# Grid manipulation in images

This notebook is made to interactively split mutliple images into n-sided blocks. This is helpful when needing to process image files in a batch of blocks, to then reconstruct them after processing.



## Imports and Installations

In [11]:
!pip install opencv-python-headless
!pip install zipfile36

In [12]:
import io
import time
from pathlib import Path
import cv2 as cv
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
import zipfile

## Functions

In [13]:
blocks = []


def draw_grid(pic, block_side=60):
    """
    Returns an OpenCV image with a n-sized block grid drawn on top
    Parameters:
      pic: Original picture to draw the grid onto (The manipulated image is copied from it)
      block_side: Side of the block (in px) - defaults to 60
    """
    k = 0
    drawable_pic = pic.copy()
    text_padding = (round(block_side * 0.05), round(block_side * 0.25))
    for i in range(0, pic.shape[1], block_side):
        for j in range(0, pic.shape[0], block_side):
            k += 1
            square = [(i, j), (i + block_side, j + block_side)]
            cv.rectangle(drawable_pic, square[0], square[1], (0, 0, 225), 2)
            cv.putText(
                drawable_pic,
                f"{k - 1}",
                (i + text_padding[0], j + text_padding[1]),
                cv.FONT_HERSHEY_SIMPLEX,
                block_side * 0.008,
                (0, 0, 225),
                1,
                cv.LINE_AA,
            )
    return drawable_pic


def get_blocks(pic, block_side=60, blocks_i=[], resize=0):
    """
    Returns a list of openCV images (blocks) from the original one passed in
    Parameters:
      pic: Original picture to get the blocks from (The manipulated images are copied from it)
      block_side: Side of the block (in px) - defaults to 60
      blocks_i: List of indices of wanted blocks from image (Get index from draw_grid()) - defaults to empty list that returns the whole original image
      resize: Side of the return block (in px) if resizing is needed - defaults to 0, that being "don't resize"
    """
    squares = []
    if pic is not None:
        for i in range(0, pic.shape[1], block_side):
            for j in range(0, pic.shape[0], block_side):
                square = [(i, j), (i + block_side, j + block_side)]
                squares.append(square)
        if len(blocks_i) == 0:
            return pic
        if resize > 0:
            return [
                cv.resize(
                    pic[
                        squares[block_i][0][1] : squares[block_i][1][1],
                        squares[block_i][0][0] : squares[block_i][1][0],
                    ],
                    (resize, resize),
                )
                for block_i in blocks_i
                if block_i < len(squares)
            ]
        else:
            return [
                pic[
                    squares[block_i][0][1] : squares[block_i][1][1],
                    squares[block_i][0][0] : squares[block_i][1][0],
                ]
                for block_i in blocks_i
                if block_i < len(squares)
            ]


def get_files_from_dir(dir_path, file_extension=".png"):
    """
    Returns a list of full path file names from inside a directory (Not recursive)
    Parameters:
      dir_path: String with the path name from which the files should be fetched from
      file_extension: Extension of files that will be fetched from directory (e.g.: .png) - defaults to ".png"
    """
    path = Path(dir_path)
    if path.exists() and path.is_dir():
        return [
            str(path / f.name)
            for f in path.iterdir()
            if (
                f.is_file()
                and len(f.name) > 3
                and f.name[-len(file_extension) :] == file_extension
            )
        ]
    else:
        return None


def handle_block_range(range):
    """
    Handles the block list when inserting multiple blocks (Separated by commas ",")
    Parameters:
      range: String with the multiple block indices (Get index from draw_grid())
    """
    set_i = range.split(",")
    try:
        return [int(i) for i in set_i]
    except ValueError:
        return None


def zip_blocks(file_extension=".png"):
    """
    Handles the zipping of the blocks global variable, then downloads the zip to machine
    Parameters:
      file_extension: Extension of files that will be zipped (e.g.: .png) - defaults to ".png"
    """
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, "w") as zippy:
        global blocks
        blocks_c = blocks.copy()
        for blocks_in_image in blocks_c:
            i = 0
            for block in blocks_in_image["img"]:
                single_img_buf = io.BytesIO()
                no_errors, encoded = cv.imencode(".png", block)
                if no_errors:
                    single_img_buf.write(encoded.tobytes())
                    single_img_buf.seek(0)
                    zippy.writestr(
                        f"{blocks_in_image['filename']}_{blocks_in_image['delta_temp'][0]}-{blocks_in_image['delta_temp'][1]}_{f'{i}'.zfill(3)}{file_extension}",
                        single_img_buf.getvalue(),
                    )
                    i += 1
    with open("blocks.zip", "wb") as writer:
        writer.write(buf.getvalue())
        files.download("/content/blocks.zip")


def select_temp():
    """
    Returns the temperature selection from original image by manual user input
    """
    min_temp = input("Insert block min temperature (°C) (e.g: 24):")
    min_temp = "#" if min_temp.strip() == "" else min_temp.strip()
    max_temp = input("Insert block max temperature (°C) (e.g: 50):")
    max_temp = "#" if max_temp.strip() == "" else max_temp.strip()
    return min_temp, max_temp


def select_crops(dir_path, file_extension=".png", block_side=60):
    """
    Handles all the process of downloading blocks from image
    Parameters:
        dir_path: String with the path name from which the files should be fetched from
        file_extension: Extension of files that will be fetched from directory (e.g.: .png) - defaults to ".png"
        block_side: Side of the block (in px) - defaults to 60
    """
    files = get_files_from_dir(dir_path, file_extension)
    for f in files:
        current_pic = cv.imread(f)
        if current_pic is not None:
            pic = draw_grid(current_pic, block_side)
            cv2_imshow(current_pic)
            time.sleep(1)
            cv.destroyAllWindows()
            min_temp, max_temp = select_temp()
            clear_output()
            cv2_imshow(pic)
            time.sleep(1)
            cv.destroyAllWindows()
            range = input("Select block(s) (e.g.: 1 / 1,2,3):")
            clear_output()
            range = handle_block_range(range)
            if range is not None and len(range) > 0:
                global blocks
                blocks.append(
                    {
                        "filename": f.split("/")[-1].replace(file_extension, ""),
                        "img": get_blocks(current_pic, block_side, range),
                        "delta_temp": (min_temp, max_temp),
                    }
                )
    zip_blocks(file_extension)

## Execution

In [14]:
select_crops('/content/files/')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>